In [2]:
from azure.ai.documentintelligence.aio import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeOutputOption, AnalyzeResult
from azure.identity import DefaultAzureCredential, AzureAuthorityHosts
from azure.core.credentials import AccessToken
import os

# Azure Government endpoint
endpoint = "https://cog-di-6pvr64gw74lwc.cognitiveservices.azure.us/"

# Configure credential for Azure Government with correct authority
credential = DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT)

# Create a wrapper credential that ensures the correct scope for Azure Government
class AzureGovCredential:
    def __init__(self, base_credential):
        self.base_credential = base_credential
    
    def get_token(self, *scopes, **kwargs):
        # Always use the Azure Government cognitive services scope
        return self.base_credential.get_token("https://cognitiveservices.azure.us/.default", **kwargs)

# Test authentication first
try:
    # Try to get a token to verify authentication
    token = credential.get_token("https://cognitiveservices.azure.us/.default")
    print(f"Authentication successful. Token expires at: {token.expires_on}")
except Exception as e:
    print(f"Authentication failed: {e}")
    raise

# Create the wrapped credential
gov_credential = AzureGovCredential(credential)

# Create client with the wrapped credential and specify a stable API version
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, 
    credential=gov_credential,
    api_version="2024-11-30"  # Use a more stable API version
)


# with open("data/role_library.pdf", "rb") as content:
#     poller = await document_intelligence_client.begin_analyze_document(
#         model_id="prebuilt-layout", 
#         analyze_request=content, 
#         content_type="application/octet-stream"
#     )
#     result: AnalyzeResult = await poller.result()
#     operation_id = poller.details["operation_id"]
    
# print(f"Document analysis completed. Operation ID: {operation_id}")
# print(f"Number of pages analyzed: {len(result.pages) if result.pages else 0}")
    

Authentication successful. Token expires at: 1749851439


In [ ]:
from azure.identity import get_bearer_token_provider

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, 
    credential=DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT),
    token_credential=get_bearer_token_provider(
                credential, "https://cognitiveservices.azure.us/.default"
            ),
    api_version="2024-11-30"  # Use a more stable API version
)



with open("data/role_library.pdf", "rb") as content:
    poller = await document_intelligence_client.begin_analyze_document(
        model_id="prebuilt-layout", 
        analyze_request=content, 
        content_type="application/octet-stream"
    )
    result: AnalyzeResult = await poller.result()
    operation_id = poller.details["operation_id"]
    
print(f"Document analysis completed. Operation ID: {operation_id}")
print(f"Number of pages analyzed: {len(result.pages) if result.pages else 0}")

ClientAuthenticationError: (None) Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.us or https://ai.azure.us), or have expired.
Code: None
Message: Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.us or https://ai.azure.us), or have expired.

In [16]:
print(result.pages)

[{'pageNumber': 1, 'angle': 0.14287519454956055, 'width': 8.5, 'height': 11, 'unit': 'inch', 'words': [{'content': 'Roles', 'polygon': [1.9969, 2.5954, 3.1435, 2.5943, 3.1416, 3.0745, 1.9966, 3.0527], 'confidence': 0.995, 'span': {'offset': 0, 'length': 5}}, {'content': 'Descriptions', 'polygon': [3.3006, 2.5957, 5.9553, 2.6048, 5.9535, 3.0835, 3.2984, 3.0757], 'confidence': 0.995, 'span': {'offset': 6, 'length': 12}}, {'content': 'at', 'polygon': [6.1045, 2.6067, 6.5244, 2.6072, 6.5244, 3.0824, 6.1025, 3.0835], 'confidence': 0.994, 'span': {'offset': 19, 'length': 2}}, {'content': 'Contoso', 'polygon': [2.1419, 3.2807, 3.8552, 3.2847, 3.8544, 3.7036, 2.1426, 3.7011], 'confidence': 0.998, 'span': {'offset': 22, 'length': 7}}, {'content': 'Electronics', 'polygon': [4.0128, 3.2861, 6.3639, 3.2908, 6.3639, 3.7036, 4.0122, 3.7036], 'confidence': 0.994, 'span': {'offset': 30, 'length': 11}}, {'content': 'Contoso', 'polygon': [6.5283, 9.8225, 7.6755, 9.8298, 7.6755, 10.0976, 6.5282, 10.0976]